# Import Libraries

In [166]:
# IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

# Data Acquisition

In [167]:
# Import dataset
VizData_TimeBrackets_Global = pd.read_csv('Viz_Data\VizData_TimeBrackets_Global.csv')
VizData_TimeBrackets_Global.rename(columns={'GEO': 'Country'}, inplace=True)
VizData_TimeBrackets_Global.head()

,Country,Time_Bucket,VALUE
0,Belgium,Zero_minutes,383.5
1,Bulgaria,Zero_minutes,83.6
2,Czechia,Zero_minutes,206.1
3,Denmark,Zero_minutes,150.1
4,Germany,Zero_minutes,1648.1


In [168]:
# Import dataset
VizData_GenderSplit_Global = pd.read_csv('Viz_Data\VizData_GenderSplit_Global.csv')

EU = ['Italy','Hungary','Norway','Estonia','Austria','Netherlands','Belgium','Germany','Portugal','Poland','Slovenia','Denmark','Ireland','France','United Kingdom','Spain','Finland','Sweden']

VizData_GenderSplit_Global["Region"] = np.where(VizData_GenderSplit_Global["Country"].isin(EU), "EU", "Other")

#VizData_GenderSplit_Global.head()


In [169]:
# Import dataset
VizData_IRE = pd.read_csv('Viz_Data\VizData_IRE.csv')
VizData_IRE.rename(columns={'Broad Industrial Group':'Group'}, inplace=True)
VizData_IRE.head()

,Group,Commute Times,Statistic Label,Population Size (1000s)
0,"Agriculture, forestry and fishing (A)",< ¼ hour,Persons at work 2016,19577.0
1,"Agriculture, forestry and fishing (A)",¼ hour - < ½ hour,Persons at work 2016,9910.0
2,"Agriculture, forestry and fishing (A)",½ hour - < ¾ hour,Persons at work 2016,5557.0
3,"Agriculture, forestry and fishing (A)",¾ hour - < 1 hour,Persons at work 2016,1307.0
4,"Agriculture, forestry and fishing (A)",1 hour - < 1½ hours,Persons at work 2016,1878.0


In [170]:
# Import dataset
country_codes = pd.read_csv("Viz_Data/country_codes.tsv", sep='\t')
country_codes = country_codes.drop('Numeric', axis=1)
country_codes.rename(columns={'Alpha-2 code': 'ISO_2'}, inplace=True)
country_codes.rename(columns={'Alpha-3 code': 'ISO_3'}, inplace=True)
country_codes.head()

,Country,ISO_2,ISO_3
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA
3,American Samoa,AS,ASM
4,Andorra,AD,AND


In [171]:
#join country in df with ISO Code for Graphing
VizData_TimeBrackets_Global = pd.merge(VizData_TimeBrackets_Global, country_codes, on='Country', how='left')

VizData_TimeBrackets_Global.rename(columns={'VALUE':'Population Size (1000s)'}, inplace=True)
VizData_TimeBrackets_Global.rename(columns={'Time_Bucket':'Commute Times'}, inplace=True)

VizData_TimeBrackets_Global.head()

,Country,Commute Times,Population Size (1000s),ISO_2,ISO_3
0,Belgium,Zero_minutes,383.5,BE,BEL
1,Bulgaria,Zero_minutes,83.6,BG,BGR
2,Czechia,Zero_minutes,206.1,CZ,CZE
3,Denmark,Zero_minutes,150.1,DK,DNK
4,Germany,Zero_minutes,1648.1,DE,DEU


In [172]:
#join country in df with ISO Code for Graphing
VizData_GenderSplit_Global = pd.merge(VizData_GenderSplit_Global, country_codes, on='Country', how='left')
VizData_GenderSplit_Global.head()

,Country,Male,Female,Both_Sexes,Region,ISO_2,ISO_3
0,China,56,38,47.0,Other,CN,CHN
1,Turkey,52,28,40.0,Other,TR,TUR
2,Korea (the Republic of),74,0,74.0,Other,KR,KOR
3,Mexico,49,24,36.5,Other,MX,MEX
4,South Africa,44,29,36.5,Other,ZA,ZAF


In [173]:
VizData_GenderSplit_EUOnly = VizData_GenderSplit_Global.loc[VizData_GenderSplit_Global['Region'] =='EU']

# Basic Visualisations

In [174]:
fig1 = px.choropleth(VizData_GenderSplit_Global,
                    locations="ISO_3", 
# colunm containing ISO 3166 country codes
                    color="Both_Sexes", 
# column by which to color-code
                    hover_name="Country", # column to display in hover information
                 color_continuous_scale=px.colors.diverging.Portland)

fig1.update_layout(
    # add a title text for the plot
    title_text = 'Average Commute Time Index (In Minutes)',
    #geo_scope = 'europe' 
# can be set to north america | south america | africa | asia | europe | usa
    geo=dict(
        showframe=False,
        #showcoastlines=False,
        projection_type='natural earth'
    ),

    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: OECD',
        showarrow = False
    )]

)

fig1.show()

In [175]:
fig2 = px.choropleth(VizData_GenderSplit_EUOnly,
                    locations="ISO_3", 
# colunm containing ISO 3166 country codes
                    color="Both_Sexes", 
# column by which to color-code
                    hover_name="Country", # column to display in hover information
                 color_continuous_scale=px.colors.diverging.Portland)

fig2.update_layout(
    # add a title text for the plot
    title_text = 'European Average Commute Time Index (In Minutes)',
    geo_scope = 'europe', 
# can be set to north america | south america | africa | asia | europe | usa


    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: OECD',
        showarrow = False
    )]

)

fig2.show()

In [176]:
#Let's add a view by Gender using the melt funtionon ouir data to create more rows which can be them leveraged by the 
#choropleth's animation_frame option. Sorting on Country and Gender allows us to 'default' the prompt to 'Both Sexes'.
VizData_GenderSplit_Global_Melt = pd.melt(VizData_GenderSplit_Global, id_vars=['Country', 'Region', 'ISO_2', 'ISO_3'], var_name="Gender", value_name="Minutes")

VizData_GenderSplit_Global_Melt.sort_values(by=["Country","Gender"],inplace=True)

VizData_GenderSplit_Global_Melt

,Country,Region,ISO_2,ISO_3,Gender,Minutes
77,Australia,Other,AU,AUS,Both_Sexes,24.5
48,Australia,Other,AU,AUS,Female,18.0
19,Australia,Other,AU,AUS,Male,31.0
70,Austria,EU,AT,AUT,Both_Sexes,29.5
41,Austria,EU,AT,AUT,Female,25.0
...,...,...,...,...,...,...
53,United Kingdom,EU,NaN,NaN,Female,16.0
24,United Kingdom,EU,NaN,NaN,Male,28.0
83,United States,Other,NaN,NaN,Both_Sexes,21.0
54,United States,Other,NaN,NaN,Female,17.0


In [177]:
fig3 = px.choropleth(VizData_GenderSplit_Global_Melt,
                    locations="ISO_3", 
# colunm containing ISO 3166 country codes
                    color="Minutes", 
# column to add to hover information
                    animation_frame="Gender",
# column by which to color-code
                    hover_name="Country", # column to display in hover information
                 color_continuous_scale=px.colors.diverging.Portland)

fig3.update_layout(
    # add a title text for the plot
    title_text = 'Average Commute Time Index by Gender (In Minutes)',
    #geo_scope = 'europe' 
# can be set to north america | south america | africa | asia | europe | usa
    geo=dict(
        showframe=False,
        #showcoastlines=False,
        projection_type='natural earth'
    ),

    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: OECD',
        showarrow = False
    )]

)

fig3.show()

# The Dashboard Section

In [178]:
import dash
from dash import dcc
#import dash_html_components as html
from dash import html
from dash.dependencies import Input, Output

from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [179]:
# Initialize the app
app = dash.Dash(__name__)

#Define the data
df1 = VizData_GenderSplit_Global_Melt
df2 = VizData_TimeBrackets_Global
df3 = VizData_IRE

In [180]:
# Define the app layout
app.layout = html.Div([
    html.H1('Commuting Index: The Interactive Dashboard'),
    html.Div([
        

        dcc.Dropdown(id='Gender-dropdown',
                     options=[{'label': c, 'value': c} for c in df1['Gender'].unique()],
                     value='Both_Sexes',
                     multi=False,
                     style={'width': '300px', 'height': '50px', 'fontSize': '20px'},
                     clearable=False),  
        dcc.Graph(id='map', figure={}),
        dcc.Dropdown(id='country-dropdown',
                     options=[{'label': c, 'value': c} for c in df1['Country'].unique()],
                     value='Ireland',
                     multi=False,
                     style={'width': '300px', 'height': '50px', 'fontSize': '20px'},
                     clearable=False),
        dcc.Dropdown(id='Industry-dropdown',
                     #options=[{'label': c, 'value': c} for c in df3['Group'].unique()],
                     options=[{"label": "Agriculture, forestry and fishing (A)", "value": "Agriculture, forestry and fishing (A)"},
                              {"label": "Mining and quarrying (B)", "value": "Mining and quarrying (B)"},
                              {"label": "Transportation and storage (H)", "value": "Transportation and storage (H)"},
                              {"label": "Total at work", "value": "Total at work"},
                              ],
                     value='Total at work',
                     multi=False,
                     style={'width': '300px', 'height': '50px', 'fontSize': '20px'},
                     clearable=False)

        
    ]),
    html.Div([
        dcc.Graph(id='bar1', figure={}),
        dcc.Graph(id='bar2', figure={}),
        dcc.Graph(id='bar3', figure={})
    ])
])

In [181]:
# Define the callback for the map
@app.callback(
    Output('map', 'figure'),
    Input('Gender-dropdown', 'value')
)
def update_map(gender):
    # Create a choropleth map of country population
    df1.a = df1[df1["Gender"] == gender]
    
    fig = px.choropleth(
        data_frame=df1.a,
        
        locations='ISO_3',
        
        color='Minutes',
        hover_data=['Country', 'Minutes'],
        color_continuous_scale=px.colors.diverging.Portland,
        labels={'Country': 'Minutes'},
        template='presentation')
    
    fig.update_layout(margin={'r': 0, 't': 0, 'l': 0, 'b': 0})
    
    # Highlight the selected country
    #fig.add_trace(go.Choroplethmapbox(geojson=countries, locations=[countries[countries['Country'] == country]['country_iso'].values[0]],
                                      #z=[1], colorscale=[[0, 'gold'], [1, 'gold']], showscale=False, marker_opacity=0.7, marker_line_width=3))
    return fig

In [182]:
# Define the callback for the bar charts
@app.callback(
    [Output('bar1', 'figure'),
     Output('bar2', 'figure'),
     Output('bar3', 'figure')
     ],
    Input('country-dropdown', 'value'),
)
def update_bars(country):
    # Filter the data by the selected country
    df_country1 = df1[df1['Country'] == country]
    df_country2 = df2[df2['Country'] == country]
    df_country3 = df3[df3['Group'] == 'Total at work']
    
    
    # Group by the gender column and get the counts
    #df_gender = df_country.groupby('Gender')['Country'].count().reset_index()
    # Create three bar charts of gender for the selected country
    fig1 = px.bar(df_country1, x='Gender', y='Minutes', color='Country', title='Country Focus: Gender Commute Differences')
    fig2 = px.bar(df_country2, x='Commute Times', y='Population Size (1000s)', color='Country', title='Country Focus: Commute Bands')
    fig3 = px.bar(df_country3, x='Commute Times', y='Population Size (1000s)', color='Group', title='Country Focus:  Commute Bands by Industrial Group')
    
    return fig1, fig2, fig3

In [183]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)